In [1]:
import json
from glob import glob

files = glob('math-qa.jsonl*.splitted.requested')
files

['math-qa.jsonl01.splitted.requested',
 'math-qa.jsonl02.splitted.requested',
 'math-qa.jsonl00.splitted.requested']

In [5]:
mapping = {}
for f in files:
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            if 'Source text\nclear\nLook up details' in l['r']['result']:
                continue
            if 'clear\nLook up details' in l['r']['result']:
                continue
            if l['r']['result'].startswith('Source text\n'):
                continue
                
            mapping[l['src']] = l['r']['result']
            
len(mapping)

105536

In [11]:
files = glob('*.parquet')
files = [f for f in files if 'mini' not in f]
files

['test.parquet', 'train.parquet', '0000.parquet']

In [13]:
import pandas as pd

df = pd.read_parquet(files[0])

In [18]:
with open('math-qa.jsonl.translated', 'w') as fopen:
    for f in files:
        df = pd.read_parquet(f)
        rows = df.to_dict(orient = 'record')
        for row in rows:
            row['Problem_ms'] = mapping.get(row['Problem'])
            row['Rationale_ms'] = mapping.get(row['Rationale'])
            row['options_ms'] = mapping.get(row['options'])
            fopen.write(f'{json.dumps(row)}\n')

/tmp/ipykernel_3525741/3733466141.py:4: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  rows = df.to_dict(orient = 'record')


In [19]:
!head -n 1 math-qa.jsonl.translated

{"Problem": "a shopkeeper sold an article offering a discount of 5 % and earned a profit of 31.1 % . what would have been the percentage of profit earned if no discount had been offered ?", "Rationale": "\"giving no discount to customer implies selling the product on printed price . suppose the cost price of the article is 100 . then printed price = 100 \u00e3 \u2014 ( 100 + 31.1 ) / ( 100 \u00e2 \u02c6 \u2019 5 ) = 138 hence , required % profit = 138 \u00e2 \u20ac \u201c 100 = 38 % answer a\"", "options": "a ) 38 , b ) 27.675 , c ) 30 , d ) data inadequate , e ) none of these", "correct": "a", "annotated_formula": "subtract(divide(multiply(add(const_100, 31.1), const_100), subtract(const_100, 5)), const_100)", "linear_formula": "add(n1,const_100)|subtract(const_100,n0)|multiply(#0,const_100)|divide(#2,#1)|subtract(#3,const_100)|", "category": "gain", "Problem_ms": "seorang pekedai menjual artikel yang menawarkan diskaun sebanyak 5 % dan memperoleh keuntungan sebanyak 31.1 % . berapaka